# Alpha-Factory Finance Demo 📈

This notebook launches a **single-node Alpha-Factory** container, activates
the *BTC/GLD momentum* strategy, then queries the FinanceAgent for **positions**
and **P&L**.  It runs **online or offline** – if `OPENAI_API_KEY` is missing the
container automatically uses a local Φ-2 model.

⚠️  **Disclaimer**: This demo is for research and educational purposes only.
It uses a simulated exchange by default and should not be used with real funds.
Nothing here constitutes financial advice.


## 1 · Parameters

In [ ]:
IMG = "ghcr.io/montrealai/alphafactory_pro:cpu-slim-latest"
PORT_API = 8000
CONTAINER = "af_nb_demo"
STRATEGY = "btc_gld"  # change to your own


## 2 · Start container

In [ ]:
import subprocess, time, requests, sys, os, json
print("⏳ Pulling image (skip if cached)…")
subprocess.run(["docker", "pull", IMG], check=True)

import socket
def port_free(port:int)->bool:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) != 0
if not port_free(PORT_API):
    raise RuntimeError(f'Port {PORT_API} is already in use')


print("🚀  Starting Alpha‑Factory …")
subprocess.Popen([
    "docker", "run", "-d", "--rm", "--name", CONTAINER,
    "-p", f"{PORT_API}:8000",
    "-e", f"FINANCE_STRATEGY={STRATEGY}",
    IMG
])

# Wait for /health
for _ in range(60):
    try:
        requests.get(f"http://localhost:{PORT_API}/health", timeout=1)
        break
    except Exception:
        time.sleep(1)
else:
    raise RuntimeError("API did not start in 60 s")

print(f"✅  API ready at http://localhost:{PORT_API}")


## 3 · Fetch positions & P&L

In [ ]:
import pandas as pd, requests, IPython.display as disp
base = f"http://localhost:{PORT_API}/api/finance"
positions = requests.get(base + "/positions").json()
pnl = requests.get(base + "/pnl").json()

disp.display(pd.json_normalize(positions).style.set_caption("Current Positions"))
disp.display(pd.json_normalize(pnl).style.set_caption("P&L (USD)"))


## 4 · Explore the trace‑graph ✨
Open [http://localhost:8088](http://localhost:8088) in your browser to watch
the planner emit decisions and tool‑calls in real time.

## 5 · Shutdown

In [ ]:
import subprocess, time
subprocess.run(["docker", "stop", CONTAINER])
print("🛑 Container stopped.")
